# Covid-19 Lab Notebook

(to edit this notebook and the associated python files, `git checkout` the corresponding commit by its hash, eg. `git checkout 422024d`)

In [1]:
from IPython.display import display, Markdown
from datetime import datetime
cur_datetime = datetime.now()
display(Markdown(f'# {cur_datetime.strftime("%d/%b/%Y %H:%M")}'))

# 24/Jun/2020 11:44

# Covid-19 model dynamics on SD

On this notebook, we'll explore and visualize multiple models of covid-19 behavior in a dynamic system approach. The models were written on cadCAD - a library for Complex Adaptive Dynamics simulations which allows you to mix and prototype 
different modelling paradigms in a reproducible and consistent manner.

In [2]:
%%capture
%matplotlib inline

# Dependences
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Experiments
import run

In [3]:
# Run all experiments. Typical run duration for an Core-i3 laptop is about 2-3min.
# Tweak the prey_predator_abm/sim_params.py file if you want it to take longer (or not).
start_time = time()
experiments = run.run()
end_time = time()
print("Execution in {:.1f}s".format(end_time - start_time))

Configurations Length: 3
Execution Method: parallelize_simulations
Execution Mode: parallelized


ValueError: too many values to unpack (expected 2)

# Covid-19 SIR model

In order to create a mathematically represent Covid-19's spread, we will use compartmental models of infectious diseases. These models assign the system's population to compartments with labels that represent different states where the individuals can be. The order of the labels usually shows the flow patterns between the compartments. The standard introductory model for it is SIR, which consists of three compartments:
- **S**usceptible: The number of individuals at risk of contracting the disease;
- **I**nfectious: The number of individuals who have been infected and are capable of infecting susceptible individuals;
- **R**ecovered: The number of individuals who have been infected and have recovered from the disease.

As SIR is a very simple model, **it considers that the disease's death rate is negligible**.

The SIR model can be expressed by the following ordinary differential equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac{d}{dt}Infected}&=\beta * Infected * \frac {Susceptible}{Total Population} -\gamma * Infected
\\{\frac {d}{dt}Recovered}&=\gamma *Infected\end{aligned}

Where the parameters are:
- $\beta$: expected amount of people an infected person infects per day
- $\gamma$: the proportion of infected recovering per day [$\gamma$ = 1/recovery time in days]



## SIR model stock and flow diagram
Before properly building the model, it's essential to graphically represent it so that our problem is conceptually understood.

The following flowchart shows how the information flows through the SIR model stocks of populations.

![covid-19 SIR model mechanism](images/covid-19_sir_model_stock_flow.png)

## Covid-19 SIR model mechanism

![covid-19 SIR model mechanism](images/covid-19_sir_model.png)

As seen in the framework above, the behavior of the SIR model are infected and recovered growth.

The infected growth causes a decrease in the suscpetible population, as previously susceptible individuals became infected; for obvious reasons, it also increases the infected population.

Similarly, the recovered growth causes a decrease in the infected population, as previously infected individuals became recovered, and it incerases the recovered population.

In [ ]:
df = experiments.dataset[0] 
plt.rcParams["figure.figsize"]=20,5
fig, ax = plt.subplots()
ax.plot(df['timestep'], df['susceptible'], label='susceptible'  )
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

# Covid-19 SEIR model

The SIR model can give us a decent analysis on the behavior of an infectious disease, but its simplicity limitates it. As many infections, Covid-19 has a significant incubation period during which individuals have been infected but neither show symptoms nor are capable of infecting other individuals. The SEIR model includes this population in compartment **E** (for Exposed).

The SEIR model is described by the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - \gamma * Infected
\\{\frac {d}{dt}Recovered}&=\gamma*Infected
\end{aligned}

Where the parameters are:
- $\beta$ expected amount of people an infected person infects per day
- $\gamma$ the proportion of infected people recovering per day ($\gamma$ = 1/D)
- $\delta$ expected rate that exposed people turn into infected

As seen above, SEIR also considers the disease's death rate negligible. It's also notable that it has very similar equations to SIR. Besides adding an equation that represents the exposed population, SEIR also has differences in the infected population equation.

While SIR considered the difference between **the amount of individuals infected by and infected person** and the amount of infected individuals recovering per day, SEIR considers the difference between **the amount of exposed individuals turned into infected** and the amount of infected individuals recovering per day.

When conceptually analyzing the meaning of each compartment, it becomes very simple to understand this change in the equation.


## Covid-19 SEIR model stock and flow diagram 
![covid-19 SEIR model mechanism](images/covid-19_seir_model_stock_flow.png)

## Covid-19 SEIR model mechanism
![covid-19 SIR model mechanism](images/covid-19_seir_model.png)

As analytically shown by the equations, the difference between both models' mechanisms is that SEIR has an exposed growth behavior that triggers a decrease in the susceptible population and an increase in the exposed population. Besides that, infected growth causes a decrease in the exposed population, differently from SIR, where it decreased the susceptible one.

In [ ]:
df = experiments.dataset[1] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

# Covid-19 SEIRD model

As we know, Covid-19 is a disease with a significant death rate (around 5% worldwide). Because of that, SIR and SEIR models can be considerably inaccurate. Therefore, the SEIRD model can better represent Covid-19, as it includes individuals who died because of the disease in compartment **D**.

The SEIRD model is based on the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - (1 - \alpha) * \gamma * Infected - \alpha * \rho * Infected
\\{\frac {d}{dt}Recovered}&= (1 - \alpha) * \gamma *Infected
\\{\frac {d}{dt}Dead}&=\alpha * \rho * Infected\end{aligned}

Where the parameters are:
- $\beta$ expected amount of people an infected person infects per day
- $\gamma$ the proportion of infected recovering per day ($\gamma$ = 1 / D)
- $\delta$ expected rate that exposed people turn into infected
- $\rho$ rate at which infected people die per day ($\rho$ = 1 / D)
- $\alpha$ death probability
- $R₀$: the total number of people an infected person infects (R₀ = β / γ)

We can again see that the equations are similar to the previous models. Besides including a death population equation, the differences occur in the infected and recovered ones, where **death rate is now considered**.

## Covid-19 SEIRD model stock and flow diagram 
![covid-19 SEIRD model mechanism](images/covid-19_seird_model_stock_flow.png)

As seen in the flowchart, now the population in the infected stock can flow to two population stocks (dead or recovered).

## Covid-19 SEIRD model mechanism 
![covid-19 SEIRD model mechanism](images/covid-19_seird_model.png)

In [ ]:
df = experiments.dataset[2] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
ax.plot(df['timestep'], df['dead'], label='dead')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

### Covid-19 SEIRD model with time variant R₀

This is the standard introductory model for Covid-19 spread. It is based on the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - (1 - \alpha) * \gamma * Infected - \alpha * \rho * Infected
\\{\frac {d}{dt}Recovered}&= (1 - \alpha) * \gamma *Infected
\\{\frac {d}{dt}Dead}&=\alpha * \rho * Infected\end{aligned}

Where the parameters are:
- $\beta$ expected amount of people an infected person infects per day
- $\gamma$ the proportion of infected recovering per day ($\gamma$ = 1 / D)
- $\delta$ expected rate that exposed people turn into infected
- $\rho$ rate at wich infected people die per day ($\rho$ = 1 / D)
- $\alpha$ death probability
- $R₀$: the total number of people an infected person infects (R₀ = β / γ)


In [ ]:
df = experiments.dataset[3] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
ax.plot(df['timestep'], df['dead'], label='dead')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

### Conclusions and final words

### Proposed challenges

#### Make R₀ time dependent


#### Variable death rate



#### Stochasticity of the parameters



#### Add real data

